# SQL Queries with Differential Privacy

## Read data
opendp.smartnoise supports issue SQL queries against CSV files, database engines, and Spark clusters.

In [1]:
import pandas as pd

pums = pd.read_csv('readers/PUMS.csv')

print(pums)

     age  sex  educ  race   income  married   pid
0     59    1     9     1      0.0        1     1
1     31    0     1     3  17000.0        0     2
2     36    1    11     1      0.0        1     3
3     54    1    11     1   9100.0        1     4
4     39    0     5     3  37000.0        0     5
..   ...  ...   ...   ...      ...      ...   ...
995   73    0     3     3  24200.0        0   996
996   38    1     2     3      0.0        0   997
997   50    0    13     1  22000.0        1   998
998   44    1    14     4    500.0        1   999
999   29    1    11     1  66400.0        0  1000

[1000 rows x 7 columns]


## Execute DP query

### Initialize PandasReader

PandasReader support to run query against Pandas dataframe. In addition to the PandasReader, opendp.smartnoise provides built-in SqlServerReader, PostgresReader, SparkReader, and PrestoReader. The Reader interface is extensible, so developers can wrap existing DB-API drivers to provide access to other popular database engines. The Reader implementations do not know anything about differential privacy, and simply return SQL query results.

### Initialize PrivateReader
The `PrivateReader` exposes the same interface as any other reader, but results will be differentially private. It works by wrapping any SQL data source that returns typed tuples.  In this sample, we wrap a `PandasReader`, which returns SQL queries against Pandas dataframes. 

In [2]:
from opendp.smartnoise.sql import PandasReader, PrivateReader
from opendp.smartnoise.metadata import CollectionMetadata

meta = CollectionMetadata.from_file('readers/PUMS.yaml')
query = 'SELECT married, COUNT(pid) AS n FROM PUMS.PUMS GROUP BY married'

reader = PandasReader(pums, meta)

private_reader = PrivateReader(reader, meta, 1.0)

result_dp = private_reader.execute_df(query)
print(result_dp)

   married    n
0    False  443
1     True  555


**Note**, in the above example, we query the PUMS microdata to get the count of individuals by marriage status.  If you run the private query repeatedly, you will see that the answer changes a bit between queries.

In [3]:
result_dp = private_reader.execute_df(query)
print(result_dp)

   married    n
0    False  449
1     True  553


The `PrivateReader` allows you to swap in differentially private results wherever exact results are currently used, it can accept some additional paramaters to control privacy/accuracy tradeoff. 

In [4]:
private_reader = PrivateReader(reader, meta, 4.0)  # large epsilon, less privacy
result = private_reader.execute_df(query)
print(result)
print()

private_reader = PrivateReader(reader, meta, 0.1)  # smaller epsilon, more privacy
result = private_reader.execute_df(query)
print(result)

   married    n
0    False  451
1     True  550

   married    n
0    False  485


## Execute normal query 
Calling the underlying `Reader` directly will give the exact result.

In [5]:
result = reader.execute_df(query)

print(result)

   married    n
0        0  451
1        1  549


## Metadata file

The `PrivateReader` needs some metadata that describes the data source.  Differentially private processing needs to know which columns can be used in numeric computations, as well as information about the sensitivity of data, and which column is the private identifier.  Metadata should be provided by the data owner, and should not be data-dependent.  For example, the acceptable range for the `age` column should be domain-specific, and should not use the actual minimum and maximum values from the data:

In [6]:
meta = CollectionMetadata.from_file('readers/PUMS.yaml')
print(meta)

PUMS.PUMS [1000 rows]
	age [int] (0,100)
	sex (card: 0)
	educ [int] (unbounded)
	race (card: 0)
	income [int] (0,500000)
	married (boolean)
	*pid [int] (unbounded)
